> Run Anaconde Prompt with Admin Rights

Get ipyleaflet help https://ipyleaflet.readthedocs.io/en/latest/index.html

Install "pip install openlocationcode"

Install "conda install gpxpy"

Get Coordinates from https://www.google.de/maps/@52.3097709,9.7330615,14z

GPS converter https://www.gpsvisualizer.com/

GeoJSON Daten http://opendatalab.de/projects/geojson-utilities/

GPX Editor Online https://www.gpxeditor.co.uk/map
GPX Tracks http://www.bikehike.co.uk/mapview.php

For GPS see http://research.ganse.org/datasci/gps/

GPS Viewer https://www.j-berkemeier.de/GPXViewer/

# Create base map for Hemmingen
Tipps: ScaleControl is not compatible with embed_html

In [11]:
# conda install -c conda-forge ipyleaflet 
## or
# conda config --add channels conda-forge
# conda install ipyleaflet 
from ipyleaflet import Map, basemaps, FullScreenControl, LayersControl, ScaleControl
from sidecar import Sidecar

def mapselection(i):
    switcher={
                0:basemaps.OpenStreetMap.Mapnik,
                1:basemaps.Esri.WorldImagery,
             }
    return switcher.get(i,"Invalid value of mapselection")

center = (52.30, 9.73)
center = (52.2995, 9.7255)
zoom = 11
basemap = mapselection(1)

m = None
m = Map(basemap=basemap,center=center, zoom=zoom, scroll_wheel_zoom=True)
m.layout.height='800px'
m.layout.width='700px'

# add controls
m.add_control(FullScreenControl(position='topleft'))
m.add_control(LayersControl(position='topleft'))

m

Map(center=[52.2995, 9.7255], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoo…

In [12]:
from ipyleaflet import basemap_to_tiles

satellit = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
m.add_layer(satellit)

# ADFC Icon and FSW Location

In [13]:
from ipyleaflet import Icon, Marker

iconPos = (52.326298, 9.676757)
icon = Icon(icon_url='img\ADFCLogo.png', icon_size=[120, 40])
markIcon = Marker(name='Icon',location=iconPos, icon=icon)
icon.icon_url

'img\\ADFCLogo.png'

In [14]:
from ipywidgets import HTML
from ipyleaflet import AwesomeIcon, Marker

locFSW = (52.319109, 9.723495)

# icon
iconGear = AwesomeIcon(
    name='gear',
    marker_color='blue',
    icon_color='darkred',
    spin=True
)

markerFSW = Marker( icon=iconGear, 
                    draggable= False,
                    location=locFSW,
                    title = 'klick mich')

# Popup Window
htmlFSW = HTML()
htmlFSW.value = '''<a href="http://adfc-hemmingen-pattensen.github.io/" 
                    target="_blank">ADFC Selbsthilfe-Werkstatt</a>'''
markerFSW.popup = htmlFSW

In [15]:
from ipywidgets import HTML
from ipyleaflet import MarkerCluster

ADFC_cluster = MarkerCluster(
    name = 'ADFC Info',
    markers=(markerFSW, markIcon)
)



m.add_layer(ADFC_cluster);
    
m

Map(center=[52.2995, 9.7255], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoo…

# RadTouren

pass GPX file

In [16]:
## Read all Route file
from os import listdir
from os.path import isfile, join

TourenPath = 'TourenHem'

Tourenfiles = [join(TourenPath,f) for f in listdir(TourenPath) if isfile(join(TourenPath, f)) if f.endswith(".gpx")]
print(len(Tourenfiles))
Tourenfiles

15


['TourenHem\\2010-01-01_91760518_Hemmingen Marienburg und zurück.gpx',
 'TourenHem\\2017-03-31_15078859_Hemmingen zum Altwarmbüchener See.gpx',
 'TourenHem\\2020-05-16_184621811_Hemmingen - Anderter Schleuse - Kronsberg.gpx',
 'TourenHem\\2020-05-17_185335550_Hemmingen Benther Berg Ronnenberg und zurück.gpx',
 'TourenHem\\2020-05-19_186353530_Hemmingen Rund um den Gerdener Berg.gpx',
 'TourenHem\\2020-05-19_186365299_Hemmingen Julius Trip Ring.gpx',
 'TourenHem\\2020-05-19_186377090_Hemmingen Leinemasch Ruthe und zurück.gpx',
 'TourenHem\\2020-05-19_186381237_Hemmingen Holtensen Lüdersen und zurück.gpx',
 'TourenHem\\2020-05-20_186945896_Hemmingen Benther Berg Ahlem und an der Ihme zurück.gpx',
 'TourenHem\\2020-05-20_186957613_Hemmingen Rethen Bockemer Holz Kronsberg und zurück.gpx',
 'TourenHem\\2020-06-03_196311070_Hemmingen Hildesheim Domäne Marienburg Barnten zurück.gpx',
 'TourenHem\\2020-06-14_202113310_Hemmingen Giftener See Hüpede.gpx',
 'TourenHem\\Der Grüne Ring.gpx',
 'Tour

In [17]:
import os.path
from os import path

def checkIfMDfileExist(gpxfile):
    mdfile = gpxfile[:-3] + 'md'
    if path.exists(mdfile):
        return mdfile
    return False

In [18]:
css = u"""
<style>
h1 {
  font-weight: bold;
  color: #000;
  font-size: 14px;
}

h2 {
  font-weight: bold;
  color: #000;
  font-size: 13px;
}
h3 {
  font-weight: bold;
  color: #000;
  font-size: 12px;
}
</style>
"""

In [19]:
# >conda install gpxpy

from ipyleaflet import  Polyline
import gpxpy
import gpxpy.gpx
import re
from markdown import markdown

#touren = []
for file in Tourenfiles:
    gpxfilefp = open(file, 'r', encoding='utf-8-sig') #use sig if file has Byte Order Mark (BOM)
    gpx = gpxpy.parse(gpxfilefp)
    gpxfilefp.close()
    
    data = [[i.latitude, i.longitude] for i in gpx.tracks[0].segments[0].points]

    match = re.search('\\\\(.+?)\.', file)
    if match:
        found = match.group(1)
    
    mdfile = checkIfMDfileExist(file)
    if mdfile:
        mdfilefp = open(mdfile, 'r', encoding='utf-8-sig') #use sig if file has Byte Order Mark (BOM)
        mdtext = mdfilefp.read()
        mdfilefp.close()
    else:
        mdtext = found
        
    # exchange link to make target = blank
    #https://www.regexpal.com/95855
    MDlink = re.compile(r"\[(.*?)\]\((.*?)\)", re.IGNORECASE) 
    subMD = r'<a href=\2 target=\"_blank\">\1</a>'
                        
    mdtext = MDlink.sub(subMD, mdtext)
                        
    colormatch = re.search(r"<!--.*Farbe:.*'(.*?)'.*-->",mdtext)
    if colormatch:
        color = colormatch.group(1)
    else:
        color = 'red'
    
    line = Polyline(
        title = found,
        name = found,
        locations=data,
        color=color,
        opacity = 0.7,
        weight =7,
        fill=False
    )


    htmlDescription = HTML()
    htmlDescription.value = css +\
                            '<div>' +\
                            markdown('Download: [GPX File]({})\n\n'.format(file) + mdtext) +\
                            '</div>'
        
    Poptext = HTML()
    Poptext.value = htmlDescription.value
    line.popup = Poptext
    line.popup_min_width = 500
    line.popup_max_height = 500

    m.add_layer(line);

m

Map(center=[52.2995, 9.7255], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoo…

In [20]:
from ipyleaflet import MeasureControl

measure = MeasureControl(
    position='topright',
    active_color = 'orange',
    primary_length_unit = 'kilometers',
    primary_area_unit = 'hectares'
)
m.add_control(measure)

measure.completed_color = 'red'


m

Map(center=[52.2995, 9.7255], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoo…

# Save as HTML and show in Browser
#https://ipywidgets.readthedocs.io/en/latest/embedding.html#python-interface

In [21]:
from ipywidgets.embed import embed_minimal_html #, dependency_state

embed_minimal_html('TourenHem.html', views=[m], title='ADFC Hem Touren')

# Start browser
!c:\DataADFC\adfc-github\adfc-hemmingen-pattensen.github.io\Touren\TourenHem.html